In [ ]:
!pip install openbb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import requests
import os
import numpy as np, pandas as pd
from datetime import datetime, timezone
from math import sqrt, exp, log
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import yfinance as yf
from openbb import obb

In [19]:
TICKER='JPM'

In [20]:
get_chains = obb.derivatives.options.chains(symbol=TICKER)

In [21]:
ALL = get_chains.to_dataframe()
ALL.rename(columns={'expiration': 'expiry'}, inplace=True)
ALL.rename(columns={'option_type':'type'}, inplace = True)

In [22]:
ASOF = datetime.now(timezone.utc).date()
ALL['asof']=pd.to_datetime(ASOF)

In [23]:
display(ALL.head(10))

,underlying_symbol,underlying_price,contract_symbol,expiry,dte,strike,type,open_interest,volume,last_trade_price,last_trade_time,bid,ask,change,change_percent,implied_volatility,in_the_money,currency,asof
0,JPM,312.05,JPM251205C00160000,2025-12-05,1,160.0,call,5,21,149.50,2025-12-03 11:15:02-05:00,150.1,153.75,1.059998,0.007141,5.333988,True,USD,2025-12-04
1,JPM,312.05,JPM251205P00160000,2025-12-05,1,160.0,put,500,500,0.01,2025-11-28 11:33:45-05:00,0.0,0.22,0.000000,0.000000,3.578126,False,USD,2025-12-04
2,JPM,312.05,JPM251205C00165000,2025-12-05,1,165.0,call,60,21,144.56,2025-12-03 11:15:02-05:00,145.1,148.75,1.069992,0.007457,5.119144,True,USD,2025-12-04
3,JPM,312.05,JPM251205P00165000,2025-12-05,1,165.0,put,501,1,0.16,2025-12-01 09:45:06-05:00,0.0,0.22,0.000000,0.000000,3.429689,False,USD,2025-12-04
4,JPM,312.05,JPM251205C00170000,2025-12-05,1,170.0,call,0,2,139.20,2025-12-03 12:09:56-05:00,140.3,143.30,-0.949997,-0.006778,4.613285,True,USD,2025-12-04
5,JPM,312.05,JPM251205C00175000,2025-12-05,1,175.0,call,58,2,134.24,2025-12-03 12:09:56-05:00,135.1,138.75,1.800003,0.013591,4.707035,True,USD,2025-12-04
6,JPM,312.05,JPM251205P00175000,2025-12-05,1,175.0,put,1,1,0.01,2025-12-02 12:31:24-05:00,0.0,0.01,-0.030000,-0.750000,2.375004,False,USD,2025-12-04
7,JPM,312.05,JPM251205C00180000,2025-12-05,1,180.0,call,1,0,130.23,2025-12-01 11:19:15-05:00,130.3,133.00,0.000000,0.000000,4.011724,True,USD,2025-12-04
8,JPM,312.05,JPM251205P00180000,2025-12-05,1,180.0,put,12,13,0.01,2025-12-03 10:31:37-05:00,0.0,0.01,-0.080000,-0.888889,2.250004,False,USD,2025-12-04
9,JPM,312.05,JPM251205C00185000,2025-12-05,1,185.0,call,1,1,123.02,2025-12-02 09:49:03-05:00,125.3,127.75,0.000000,0.000000,3.623048,True,USD,2025-12-04


In [24]:
try:
    ALL.to_parquet('/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/combined_options_data.parquet', index=False)
    print("Successfully saved combined_options_data.parquet to Google Drive.")
except Exception as e:
    print(f"Error saving file to Google Drive: {e}")
    print("Make sure you have mounted your Google Drive correctly.")

Successfully saved combined_options_data.parquet to Google Drive.


In [25]:
# Underlying historical prices (for realised vol, returns, etc.)
hist_price = yf.Ticker(TICKER).history(period="2y")  # choose horizon
hist_price_path = '/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet'
hist_price.to_parquet(hist_price_path)
print(f"Saved underlying history to {hist_price_path}")


Saved underlying history to /content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet
